In [1]:
import torch, casadi_on_gpu as cog

kernels = {k["function_name"]: k for k in cog.list_kernels()}
fk = kernels["fkeval"]
print(fk)

{'function_name': 'fkeval', 'kernel_name': 'fkeval_kernel', 'batch_inputs': [0], 'input_nnz': [4, 6, 6], 'output_nnz': [6]}


In [2]:
N = 80000
q_all = torch.zeros((N, fk["input_nnz"][0]), device="cuda", dtype=torch.float32)
p1 = torch.zeros((fk["input_nnz"][1],), device="cuda", dtype=torch.float32)
p2 = torch.zeros((fk["input_nnz"][2],), device="cuda", dtype=torch.float32)
out = torch.zeros((N, fk["output_nnz"][0]), device="cuda", dtype=torch.float32)
stream = torch.cuda.current_stream().cuda_stream

cog.launch(
    "fkeval",
    [q_all.data_ptr(), p1.data_ptr(), p2.data_ptr()],
    [out.data_ptr()],
    N,
    stream_ptr=stream,
    sync=True,
)

print(torch.isfinite(out).all())
print(out)

tensor(True, device='cuda:0')
tensor([[-8.0000e-02,  5.3072e-08,  3.4450e-02,  0.0000e+00,  0.0000e+00,
         -1.0472e+00],
        [-8.0000e-02,  5.3072e-08,  3.4450e-02,  0.0000e+00,  0.0000e+00,
         -1.0472e+00],
        [-8.0000e-02,  5.3072e-08,  3.4450e-02,  0.0000e+00,  0.0000e+00,
         -1.0472e+00],
        ...,
        [-8.0000e-02,  5.3072e-08,  3.4450e-02,  0.0000e+00,  0.0000e+00,
         -1.0472e+00],
        [-8.0000e-02,  5.3072e-08,  3.4450e-02,  0.0000e+00,  0.0000e+00,
         -1.0472e+00],
        [-8.0000e-02,  5.3072e-08,  3.4450e-02,  0.0000e+00,  0.0000e+00,
         -1.0472e+00]], device='cuda:0')


In [3]:
bad_mask = ~torch.isfinite(out)
bad_rows = bad_mask.any(dim=1).nonzero().squeeze()

print("num bad rows:", bad_rows.numel())
print("first bad rows:", bad_rows.tolist())

# bad rows are as a result of invalid combination of sim parameters

num bad rows: 0
first bad rows: []


In [4]:
# Warm up
warmup = 50
for _ in range(warmup):
    cog.launch(
        "fkeval",
        [q_all.data_ptr(), p1.data_ptr(), p2.data_ptr()],
        [out.data_ptr()],
        N,
        stream_ptr=stream,
        sync=False,
    )
torch.cuda.synchronize()

# Time
reps = 500
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
for _ in range(reps):
    cog.launch(
        "fkeval",
        [q_all.data_ptr(), p1.data_ptr(), p2.data_ptr()],
        [out.data_ptr()],
        N,
        stream_ptr=stream,
        sync=False,
    )
end.record()
torch.cuda.synchronize()

ms_total = start.elapsed_time(end)
ms_per = ms_total / reps
throughput = (N / ms_per) * 1000.0
print(f"4 dof forward kinematic model [Pytorch]")
print(f"FK total: {ms_total:.3f} ms for {reps} calls")
print(f"FK per call: {ms_per:.4f} ms")
print(f"Throughput: {throughput:,.0f} eval/s (batch N={N})")


4 dof forward kinematic model [Pytorch]
FK total: 5.195 ms for 500 calls
FK per call: 0.0104 ms
Throughput: 7,699,509,591 eval/s (batch N=80000)


In [5]:
import cupy as cp
import casadi_on_gpu as cog

kernels = {k["function_name"]: k for k in cog.list_kernels()}
fk = kernels["fkeval"]

N = 80000
q_all = cp.zeros((N, fk["input_nnz"][0]), dtype=cp.float32)
p1 = cp.zeros((fk["input_nnz"][1],), dtype=cp.float32)
p2 = cp.zeros((fk["input_nnz"][2],), dtype=cp.float32)
out = cp.zeros((N, fk["output_nnz"][0]), dtype=cp.float32)

stream = cp.cuda.get_current_stream().ptr
cog.launch(
    "fkeval",
    [q_all.data.ptr, p1.data.ptr, p2.data.ptr],
    [out.data.ptr],
    N,
    stream_ptr=stream,
    sync=False,
)
out


array([[-7.9999998e-02,  5.3071794e-08,  3.4449995e-02,  0.0000000e+00,
         0.0000000e+00, -1.0472052e+00],
       [-7.9999998e-02,  5.3071794e-08,  3.4449995e-02,  0.0000000e+00,
         0.0000000e+00, -1.0472052e+00],
       [-7.9999998e-02,  5.3071794e-08,  3.4449995e-02,  0.0000000e+00,
         0.0000000e+00, -1.0472052e+00],
       ...,
       [-7.9999998e-02,  5.3071794e-08,  3.4449995e-02,  0.0000000e+00,
         0.0000000e+00, -1.0472052e+00],
       [-7.9999998e-02,  5.3071794e-08,  3.4449995e-02,  0.0000000e+00,
         0.0000000e+00, -1.0472052e+00],
       [-7.9999998e-02,  5.3071794e-08,  3.4449995e-02,  0.0000000e+00,
         0.0000000e+00, -1.0472052e+00]], dtype=float32)

In [6]:
stream = cp.cuda.get_current_stream().ptr

# Warmup
warmup = 50
for _ in range(warmup):
    cog.launch(
        "fkeval",
        [q_all.data.ptr, p1.data.ptr, p2.data.ptr],
        [out.data.ptr],
        N,
        stream_ptr=stream,
        sync=False,
    )
cp.cuda.runtime.deviceSynchronize()

# Timing
reps = 500
start = cp.cuda.Event()
end = cp.cuda.Event()

start.record()
for _ in range(reps):
    cog.launch(
        "fkeval",
        [q_all.data.ptr, p1.data.ptr, p2.data.ptr],
        [out.data.ptr],
        N,
        stream_ptr=stream,
        sync=False,
    )
end.record()

end.synchronize()

ms_total = cp.cuda.get_elapsed_time(start, end)
ms_per = ms_total / reps
throughput = (N / ms_per) * 1000.0
print(f"4 dof forward kinematic model [Cupy]")
print(f"FK total: {ms_total:.3f} ms for {reps} calls")
print(f"FK per call: {ms_per:.4f} ms")
print(f"Throughput: {throughput:,.0f} eval/s (batch N={N})")


4 dof forward kinematic model [Cupy]
FK total: 5.153 ms for 500 calls
FK per call: 0.0103 ms
Throughput: 7,762,914,743 eval/s (batch N=80000)
